In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 7.5MB/s 
     |████████████████████████████████| 92kB 5.1MB/s 
     |████████████████████████████████| 133kB 12.0MB/s 
     |████████████████████████████████| 6.7MB 11.1MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
ERROR: botocore 1.19.8 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [3]:
pip install urllib3==1.25.10

     |████████████████████████████████| 133kB 5.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from statistics import mode

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [22]:
def train_validate_test_split(df, train_percent=.7, validate_percent=.1, seed=42):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

def sample_data(df,sample,seed):
    X_train, _, y_train, _ = train_test_split( df['tweet'], df['label'], train_size=sample, random_state=seed, stratify=df['label'])
    return pd.concat([X_train,y_train], axis = 1 )

def tokenize_data(df):
    sentences = ["[CLS] " + query + " [SEP]" for query in df['tweet']]
    # Tokenize with multilingual BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128

    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return input_ids, attention_masks

def Data_Loader(inputs_ids, attention_masks, df,batch_size=16): 
    data = TensorDataset(torch.LongTensor(inputs_ids), torch.LongTensor(attention_masks), torch.LongTensor(df['label'].values))
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

In [23]:
def model_train(model, train_dataloader, validation_dataloader):
    # Store our loss and accuracy for plotting
    train_loss_set = []
    # BERT training loop
    epochs = 3
    for _ in trange(epochs, desc="Epoch"):  
        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
        print("Train loss: {}".format(tr_loss/nb_tr_steps))

        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1
        validation_accuracy = (eval_accuracy/nb_eval_steps)
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    return validation_accuracy

In [24]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_test(model,prediction_dataloader):
    model.eval()
    # Tracking variables 
    predictions , true_labels = [], []
# Predict 
    for batch in prediction_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions+=list(np.argmax(logits, axis=1).flatten())
        true_labels+=list(label_ids.flatten())
        # Store predictions and true labels
#         predictions.append(logits)
#         true_labels.append(label_ids)
    test_f1_score = f1_score(true_labels, predictions, average= 'macro')
    print("Macro F1 Score:",test_f1_score)
    test_accuracy_score = accuracy_score(true_labels, predictions)
    print("Accuracy score:", test_accuracy_score, "\n")
    print("="*100)

In [25]:
def model_initialise():
  # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
  model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=2)
  model.cuda()

  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
  ]

  optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-5, warmup=.1)
  return model, optimizer

In [38]:
languages = ['Arabic','French','English']
directory = './'
drive_directory = '/content/drive/My Drive/mBERT_models'
for lang in languages:
	df = pd.read_csv(os.path.join(directory, lang+'.csv'))
	sample_sizes = [16, 32, 64, 128, 256]
	for sample in sample_sizes:
		best_validation_score = 0  
		seeds = [2018,2019, 2020, 2021, 2022]
		for seed in seeds:
			train_df, validation_df, test_df = train_validate_test_split(df)
			train_len = len(train_df)
			if sample==256 and seed==2022:
			    sample_sizes.append(train_len)
			if sample == train_len and seed == 2022:
			    sample_sizes.remove(train_len)
			model, optimizer = model_initialise()
			if(sample != train_len):  
				train_df_sample = sample_data(train_df,sample,seed)
				train_input_ids, train_attention_masks = tokenize_data(train_df_sample)
				train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df_sample)
			else:
				train_input_ids, train_attention_masks = tokenize_data(train_df)
				train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df)

			validation_input_ids, validation_attention_masks = tokenize_data(validation_df)
			validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_df)
			print("\nModel Summary:")
			print('Language:', lang)
			print('Sample Size:', sample)
			print('Seed value:', seed)
			validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
			if(validation_accuracy > best_validation_score):
				best_validation_score = validation_accuracy
				print('Language:', lang)
				print("The Best Seed for the sample ",sample, " is : ",seed)
				fname = "mBERT_"+str(lang)+"_"+str(sample)+'.pth'
				path = os.path.join(drive_directory, fname)
				torch.save(model.state_dict(), path)
			test_input_ids, test_attention_masks = tokenize_data(test_df)
			test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_df)
			model_test(model, test_dataloader)

t_total value of -1 results in schedule not being applied

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


Model Summary:
Language: Arabic
Sample Size: 16
Seed value: 2018
Train loss: 0.7154797315597534



Epoch:  33%|███▎      | 1/3 [00:01<00:03,  1.77s/it]

Validation Accuracy: 0.7905405405405406
Train loss: 0.5840165019035339



Epoch:  67%|██████▋   | 2/3 [00:03<00:01,  1.77s/it]

...


Epoch:  67%|██████▋   | 2/3 [35:24<17:42, 1062.64s/it]

Validation Accuracy: 0.854582159115543
Train loss: 0.37307838160676265



Epoch: 100%|██████████| 3/3 [53:04<00:00, 1061.65s/it]

Validation Accuracy: 0.8226208541079558


Macro F1 Score: 0.7443128113333295
Accuracy score: 0.8237219007872522 



In [ ]:
# languages = ['French','Arabic','English']
# directory = './'
# drive_directory = '/content/drive/My Drive/mBERT_models'
# for lang in languages:
#     df = pd.read_csv(os.path.join(directory, lang+'.csv'))
#     sample_sizes = [16, 32, 64, 128, 256]
#     for sample in sample_sizes:
#             best_validation_score = 0  
#             seeds = [2018,2019, 2020, 2021, 2022]
#             for seed in seeds:
#                 model, optimizer = model_initialise()
#                 train_df, validation_df,test_df = train_validate_test_split(df, seed =seed)
#                 train_len = len(train_df)
#                 if sample==256 and seed==2022:
#                   sample_sizes.append(train_len)
#                 if sample == len(train_df) and seed == 2022:
#                   sample_sizes.remove(train_len)
#                 if(sample != train_len):  
#                   train_df = sample_data(train_df,sample ,seed)
#                 train_input_ids, train_attention_masks = tokenize_data(train_df)
#                 validation_input_ids, validation_attention_masks = tokenize_data(validation_df)
#                 train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df)
#                 validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_df)
#                 print("\nModel Summary:")
#                 print('Language:', lang)
#                 print('Sample Size:', sample)
#                 print('Seed value:', seed)
#                 validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
#                 if(validation_accuracy > best_validation_score):
#                     best_validation_score = validation_accuracy
#                     print('Language:', lang)
#                     print("The Best Seed for the sample ",sample, " is : ",seed)
#                     fname = "mBERT_"+str(lang)+"_"+str(sample)+'.pth'
#                     path = os.path.join(drive_directory, fname)
#                     torch.save(model.state_dict(), path)
#                 test_input_ids, test_attention_masks = tokenize_data(test_df)
#                 test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_df)
#                 model_test(model, test_dataloader)

In [37]:
languages = ['Arabic','French','English']
directory = './'
drive_directory = '/content/drive/My Drive/mBERT_models'
for lang in languages:
  df = pd.read_csv(os.path.join(directory, lang+'.csv'))
  best_validation_score = 0  
  seeds = [2018,2019, 2020, 2021, 2022]
  for seed in seeds:
	  model, optimizer = model_initialise()
	  train_df, validation_df,test_df = train_validate_test_split(df, seed =seed)
	  train_input_ids, train_attention_masks = tokenize_data(train_df)
	  validation_input_ids, validation_attention_masks = tokenize_data(validation_df)
	  train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df)
	  validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_df)
	  print("\nModel Summary:")
	  print('Language:', lang)
	  print('Sample Size:', len(train_df))
	  print('Seed value:', seed)
	  validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
	  if(validation_accuracy > best_validation_score):
		  best_validation_score = validation_accuracy
		  print('Language:', lang)
		  print("The Best Seed for the sample ",len(train_df), " is : ",seed)
		  fname = "mBERT_"+str(lang)+"_"+str(len(train_df))+'.pth'
		  path = os.path.join(drive_directory, fname)
		  torch.save(model.state_dict(), path)
	  test_input_ids, test_attention_masks = tokenize_data(test_df)
	  test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_df)
	  model_test(model, test_dataloader)

t_total value of -1 results in schedule not being applied

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


Model Summary:
Language: Arabic
Sample Size: 16
Seed value: 2018
Train loss: 0.4661037065382079



Epoch:  33%|███▎      | 1/3 [00:58<01:57, 58.71s/it]

Validation Accuracy: 0.7162162162162162
Train loss: 0.40474535656843597



Epoch:  67%|██████▋   | 2/3 [01:57<00:58, 58.73s/it]

...

Macro F1 Score: 0.7989439629247415
Accuracy score: 0.8817946505608283 



t_total value of -1 results in schedule not being applied

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


Model Summary:
Language: Arabic
Sample Size: 16
Seed value: 2022


Epoch:   0%|          | 0/3 [00:01<?, ?it/s]


KeyboardInterrupt: ignored